In [ ]:
{
  // Utility code
  // Hack to get WebSocket into global on Node
  ;(global as any).WebSocket = require('isomorphic-ws')
}

# The Hub

## Threads Services

Threads Services (e.g. pinning encrypted data on IPFS and helping multiple peers relay updates across the network) can be built and deployed to the network using [go-threads](https://github.com/textileio/go-threads). Textile offers a number of these functions through [The Hub](), which can help developers scale their applications on IPFS. Attaching the Hub to your databases will allow developers to deliver high-quality user-experiences. In order to make this as straightforward as possible, developers looking to leverage the remote Hub need to understand a few additional basic concepts.

### API Keys

The Hub APIs are available for your apps and your app users. You can use the Hub APIs with a privileged [Account API Key]() or with a [User Key](). Both have the ability to push new data to a ThreadsDB, and relay ThreadsDB updates (among other things).

#### Account Key

Account keys provide direct access to developer and org account Buckets and Threads. Account keys make it possible to build apps that have full access to developer or organization Buckets. You can use account keys to integrate your Buckets into CI, dashboards, team messaging integration, etc. To create a new Account Key use `tt key create` and select the `account` option.

#### User Key

User keys provide existing external identities (users) access to their own threads, under the custodianship of the parent (developer) account. Apps can persist and replicate ThreadsDB data for users. A single user key can be added to your app to authenticate many users to your Hub resources. To create a new User Key use `tt key create` and select the `user` option.

### Data Ownership

The databases you create over the APIs are owned in one of three ways.

1. Developer owned. If you use an account key with the Buckets or ThreadsDB APIs, the data will be linked directly to your account.
2. Org owned. If you create an account key using the --org flag, the Buckets and Threads will be linked to the organization.
3. User owned. If you create a user key, Textile allows your app to provision new Buckets and Threads on behalf of your users. This data will be signed and owned by your end-users and only accessible to them. 

Developers can specify the *context* to customize exactly *how* resources (e.g., storage, networking, etc) are used/allocated within their own apps, and which of the three above ownership strategies are applied. This Conext API allows a developer to shift what role they are using to access the remote Hub APIs. This is interesting because a developer is able to access Hub resources as themselves (i.e., the developer), with all the administrative capabilities that entails, or as users of their app, which are sandboxed but able to create Threads (and Buckets) of their own *within* that user-scoped sandbox.

This is a very powerful framework for accessing and allocating developer resources on behalf of users, while still providing the control and quality user-experience that apps built on Threads should provide.

### Identity

Related to dadta ownership is the concept of identity. Textile's Hub and Buckets/ThreadsDB APIs are flexible when it comes to user identity, allowing you to handle user identities (for access control and security/encryption) in the best way for your app and your users. In order to handle *multiple* peers collaborating on a single database, as well as the ability to handle storage *on behalf* of a user, Hub APIs expect a simple Identity interface for singing and validating updates.

In [ ]:
interface Public {
  verify(data: Buffer, sig: Buffer): Promise<boolean>
  toString(): string
  bytes: Buffer
}

interface Identity {
  sign(data: Buffer): Promise<Buffer>
  public: Public
}

Identity here represents any entity capable of signing a message. This is a simple [public key infrastructure](https://en.wikipedia.org/wiki/Public_key_infrastructure) inspired interface that similarly requires the implementer to be capable of returning an associated public key for verification. In many cases, the Identity will just be a private key, but callers can use any setup that suits their needs. A default implementation based on [Libp2p's crypto library]() is provided for convinience (and is also used by default if no identity is provided), however, many developers will want to use alternative identity provides, such as [3box/Ceramic](), [Fortmatic](), and existing private/public keypair, or a web3 provider such as [Metamask](). Textile Hub also provides email-based identities.

Identities also provide a way for developers to allocate resources (i.e., storage) for a particular user, and in fact, is a key component in ensuring that a user *controls their own data*.

### Example: User Owned Database

To illustrate the utility of Identity and Context, in the following example, we will create a user owned ThreadsDB within a "user context". This should provide a useful example for getting started with Textile's Hub APIs in the context of a database. We'll also interact with our remote database using the Threads Client library (see also additional examples using the [local-first Database]() library).

To get started with Textile's Context API, follow the instructions in our [getting started guide](https://docs.textile.io/hub/introduction/). Once you have downloaded and installed the command-line tools, be sure to create a developer account.

Next, create a new `user` key using `tt key create` command line tool. After some steps to create an account, you can create the keys.


```bash
➜ tt key create # select the 'user' option
```


This should produce output similar to the following. Make note of these values, but do not share them! For a Node app, it is a good idea to use a tool such as [dotenv](https://github.com/motdotla/dotenv) to reference them in your apps, which is what we will assume below. So create a simple `.env` file with `USER_API_KEY` and `USER_API_SECRET` keys.

```bash
✔ user

  KEY                          SECRET                          TYPE  
  bqab5csdh...no6jjezox4       bm2tk476yivwlw...3a4cayll7ztha  user  

> Success! Created new API key and secret
```


Once you have the key information handy, let's jump to some code. We'll start with the required imports, and initialize a basic default `Context`.

In [ ]:
import { Context } from '@textile/context'
import { Libp2pCryptoIdentity } from '@textile/threads-core'
import { Client } from '@textile/threads-client'
import { config } from 'dotenv'

// Load your .env into process.env
const parsed = config()

### Prepare Identity & Token

The `Context` module provides a useful set of methods for managing and using developer keys and signatures on the client side, abstracting away some of the complexity of developer key management and communication with Textile's remote Hub gRPC APIs.

For example, to create a new basic Context that will attempt to connect to Textile's Hub staging server, you can initialize a Context by specifying the remote host url.

While we're at it, we'll also create a default identity for our user, using the default `Libp2pCryptoIdentity` object. In practice, you might have your own identity provider, or you might want to use a hierarchical key/wallet or menmonic phrase to help store a users keys for them. Whatever you decide, Textile's generic Identity interface should be able to support it.

In [ ]:
const ctx = new Context()
const identity = await Libp2pCryptoIdentity.fromRandom() // Random identity

The next step is to authenticate the user with your User Key and Secret. This will allow the user to store threads and buckets using your developer resources on the Hub.

In [ ]:
// Update the context WITH the user key information
await ctx.withUserKey({
   key: process.env.USER_API_KEY,
   secret: process.env.USER_API_SECRET,
   type: 1, // User key type
 })
// This will also return a promise with your updated context

### Setup ThreadDB

Now we will connect to our remote ThreadsDB client. This will allow us to connect to a remote ThreadsDB on the Textile Hub. This is an alternative to a local-first database where keys and data are stored _locally_ on the device (see local first [database example]() for details on this).

In [ ]:
const db = new Client(ctx)
// API calls will now include the credentials created above

With the ThreadsDB instance ready to connect to the remote database, it is time to generate a user token. This allows us (the developer) to allocate user-scoped resources without our remote database. The app user (defined by their Identity created above) needs an API token to perform database operations. The API will give you one based on ID plus your developer credentials. The token will be added to the existing db.context. The token can also manually be added to a context if one has already been generated for the user.

In [ ]:
const token = await db.getToken(identity)
// Or, update it manually from an existing token
// db.context.withToken(token)

Note that the context is actually reusable in future sessions (except it will likely expire and need to be updated). It can base stored in a session or however your app stores app state (securely).

In [ ]:
JSON.stringify(ctx.toJSON(), undefined, 2)

An app should create a minimal number of Threads per user to avoid creating unnecessary storage. A single Thread can contain a large number of distinct Collections for different types of data. Here, we create a new Thread for a user, but you could similarly store this ThreadID in (say) a local database or your app state and restore it using `ThreadID.fromString()`.

In [ ]:
import { ThreadID } from '@textile/threads-id'
const id = ThreadID.fromRandom()

// Each new ThreadID requires a `newDB` call.

await db.newDB(id) // Updates the context to include the thread id
// Or, do it manually
// db.context.withThread(id)

For this example, we're going to be working with a Collection of Astronauts. The schema looks like this.

In [ ]:
const astronautSchema = {
   $id: 'https://example.com/astronaut.schema.json',
   $schema: 'http://json-schema.org/draft-07/schema#',
   title: 'Astronauts',
   type: 'object',
   required: ['_id'],
   properties: {
     _id: {
       type: 'string',
     },
     firstName: {
       type: 'string',
     },
     lastName: {
       type: 'string',
     },
     missions: {
       type: 'integer',
       minimum: 0,
     },
   },
 };

Using that schema, we'll create a new collection. See [this other example]() for details about Collections, Schemas, and Instances.

In [ ]:
await db.newCollection(id, 'Astronaut', astronautSchema)

### Add Instance to Collection

Now that our ThreadDB contains the Astronaut Collection, you just need to add a new astronaut that matches the expected schema. If you run the following code many times, you'll notice many Buzz Aldrin entries in your ThreadDB, each with a unique ID.

In [ ]:
const ids = await db.create(id, 'Astronaut', [
  {
    _id: '', // Leave empty to auto-generate
    firstName: 'Buzz',
    lastName: 'Aldrin',
    missions: 2,
  }
])

### Query from our Collection

You can search all your existing Buzz Aldrins pretty easily. You can also clean up our entries (just delete them all!), modify them, or perform various transactions.

In [ ]:
import { Where } from '@textile/threads-client'

const q = new Where('firstName').eq('Buzz')
const r = await db.find(id, 'Astronaut', q)

// Extract just the ids
const ids = r.instancesList.map((instance: any) => instance._id)

console.log(`Found ${ids.length} entries`)

// Cleanup!
await db.delete(id, 'Astronaut', ids)

We leave the remaining operations as an exercise for the reader. Have fun, explore, and let us know what you think!